In [1]:
import cufflinks as cf
import pandas as pd
import pymongo

In [2]:
cf.go_offline()

In [3]:
myclient = pymongo.MongoClient("mongodb://mongo:27017/", username="root", password="root")
bili_db = myclient['bili_data']
def read(source, filter):
    coll_curr = bili_db[source]
    result = pd.DataFrame([item for item in coll_curr.find(filter)])
    if not result.empty:
        result.drop(columns="_id", inplace=True)
    return result

In [4]:
upid = 19642758
df_data = pd.DataFrame(read("up_video_list", {"mid": upid}))
df_data['created'] = pd.to_datetime(df_data['created'], unit='s')

ServerSelectionTimeoutError: mongo:27017: [Errno 11001] getaddrinfo failed, Timeout: 30s, Topology Description: <TopologyDescription id: 60da823132a695554c1242cc, topology_type: Single, servers: [<ServerDescription ('mongo', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('mongo:27017: [Errno 11001] getaddrinfo failed')>]>

In [ ]:
df_data

In [ ]:
df1 = df_data[['created', 'stat#favorite', 'stat#coin', 'stat#like']]
df2 = df1.groupby([df1.created.dt.year, df1.created.dt.month]).agg('sum')
df2.iplot(kind='bar', barmode='stack',title='视频数据按月统计', xTitle='时间', yTitle='数量')

In [ ]:
df3 = df_data[['created', 'stat#favorite', 'stat#coin', 'stat#like', 'play']]
df3 = df3.groupby([df3.created.dt.year, df3.created.dt.month]).mean()
df3['like_rate'] = df3['stat#like'] / df3['play']
df3['favorite_rate'] = df3['stat#favorite'] / df3['play']
df3['coin_like_rate'] = df3['stat#coin'] / df3['stat#like']
df3 = df3[['like_rate', 'favorite_rate', 'coin_like_rate']]

In [ ]:
df3.iplot(kind='lines', orientation='v',
          title='up视频质量走势参考', xTitle='时间', yTitle='比率')

In [ ]:
df3.div(df3.iloc[0,:]).iplot(kind='lines', orientation='v',
          title='up视频质量走势参考(加权)', xTitle='时间', yTitle='比率')

In [ ]:
df4 = df_data[['created',  'stat#favorite', 'stat#coin', 'stat#like', 'play']]
df5 = df4.iloc[5:-5]
df5.avg('play')

In [ ]:
df_data